In [33]:
from pymol import cmd
import numpy as np
import os
cif_file = "/data/home/mrichte3/RNASeq/output_cifs_overlap/ENSG00000005073.cif"
cmd.reinitialize()
cmd.load(cif_file, 'PIK3CB_guide_target') 
cmd.select("phosphate_atoms", "chain B and resi 1 and (name OP1 or name OP2 or name P)")
cmd.extract("phosphate_group", "phosphate_atoms")
cmd.remove("phosphate_atoms")
cmd.select("phosphate_atoms", "chain C and resi 1 and (name OP1 or name OP2 or name P)")
cmd.extract("phosphate_group", "phosphate_atoms")
cmd.remove("phosphate_atoms")
output_dir = '/data/home/mrichte3/RNASeq/unmod'
basename = os.path.splitext(os.path.basename(cif_file))[0]
cmd.save(f"{basename}.pdb", 'PIK3CB_guide_target')


In [ ]:
def run_mini(command, input_text=None):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    print(output)
    for line in output.splitlines():
        if ("steepest descents converged to" in line.lower() or
            # "fatal" in line.lower() or
            # "error" in line.lower() or
            "largest distance between excluded atoms:" in line.lower() or
            "inf, atom= " in line.lower() or
            "steepest descents did not converge" in line.lower()):
            print(line, file=sys.stderr)
            if "inf, atom= " in line.lower():
                atom_match = re.search(r'inf, atom= (\d+)', line)
                if atom_match:
                    atom = int(atom_match.group(1))
                    print(atom)
                    return f"Error at atom: {atom}"
            match = re.search(r'(\d+) steps', line)
            if match:
                steps = int(match.group(1))
                return steps == 5001
def run_command(command, input_text=None, max_chars=100):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    print(output)
    for line in output.splitlines():
        if ("largest distance between excluded atoms" in line.lower() or 
            "largest distance between excluded atoms:" in line.lower() or
            "steepest descents did not converge" in line.lower() or
            "error 1 [file ions.mdp]" in line.lower()):
            print(line.strip(), file=sys.stderr)

# command = ["gmx", "make_ndx", "-f", "structure_solv_ions.gro", "-o", "index.ndx"]
# input_text = "name 19 SOLV\n1 | 12\nname 20 SOLU\nq\n"
# run_command(command, input_text)

command = ["gmx", "grompp", "-v", "-f", f"step4.0_minimization.mdp", "-o", f"step4.0_minimization.tpr", 
           "-c", f"structure_solv_ions.gro", "-r", f"structure_solv_ions.gro", 
           "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
run_mini(command)
command = ["gmx", "mdrun", "-v", "-deffnm", "step4.0_minimization", "-ntmpi", "1"]
tries = 0
while tries < 3 and not run_mini(command): ###
    tries += 1

In [5]:
import subprocess
import os
import shutil
import time
import re
import sys


os.environ['PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/bin:' + os.environ['PATH']
if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib:' + os.environ['LD_LIBRARY_PATH']
else:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib'
os.environ['GMX_MAXBACKUP'] = '-1'
# os.environ['GMX_MAXCONSTRWARN'] = '-1'

# if len(sys.argv) != 2:
#     print("Usage: python script.py <gpu_index>", flush=True)
#     sys.exit(1)
# gpu_index = int(sys.argv[1])
gpu_index = 0
num_gpus = 6
pdb_directory = '/data/home/mrichte3/RNASeq/amide/'
# pdb_directory = '/data/home/mrichte3/RNASeq/gna/'



# def run_command(command, input_text=None, max_chars=100):
#     result = subprocess.run(command, capture_output=True, text=True, input=input_text)
#     output = result.stdout + result.stderr
#     print(output)
#     for line in output.splitlines():
#         # if ("warning" in line.lower() or "fatal" in line.lower() or "random" in line.lower() or 
#         if ("largest distance between excluded atoms:" in line.lower()):
#             print(line, file=sys.stderr)
def run_command(command, input_text=None, max_chars=100):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if ("largest distance between excluded atoms" in line.lower() or 
            "largest distance between excluded atoms:" in line.lower() or
            "steepest descents did not converge" in line.lower() or
            "error 1 [file ions.mdp]" in line.lower()):
            print(line.strip(), file=sys.stderr)

def run_mini(command, input_text=None):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if ("steepest descents converged to" in line.lower() or
            # "fatal" in line.lower() or
            # "error" in line.lower() or
            "largest distance between excluded atoms:" in line.lower() or
            "inf, atom= " in line.lower() or
            "steepest descents did not converge" in line.lower()):
            print(line, file=sys.stderr)
            if "inf, atom= " in line.lower():
                atom_match = re.search(r'inf, atom= (\d+)', line)
                if atom_match:
                    atom = int(atom_match.group(1))
                    print(atom)
                    return f"Error at atom: {atom}"
            match = re.search(r'(\d+) steps', line)
            if match:
                steps = int(match.group(1))
                return steps == 5001
    return False
    
def run_stucture_setup(input_pdb):
    rm_command = "rm *.gro"
    subprocess.run(rm_command, shell=True)
    rm_command = "rm *.tpr"
    subprocess.run(rm_command, shell=True)
    command = ["gmx", "pdb2gmx", "-f", f"{input_pdb}", "-o", "structure_processed.gro", 
               "-p", "topol.top", "-i", "posre.itp"]
    input_text = "6\n1\n"        ############ 6 1 for custom
    run_command(command, input_text)
    command = ["gmx", "editconf", "-f", "structure_processed.gro", "-o", "structure_box.gro", "-c", "-d", "1.0", "-bt", "cubic"]
    run_command(command)
    command = ["gmx", "solvate", "-cp", "structure_box.gro", "-cs", "spc216.gro", "-o", "structure_solv.gro", "-p", "topol.top"]
    run_command(command)
    ###########################fails
    command = ["gmx", "grompp", "-f", "ions.mdp", "-c", "structure_solv.gro", "-p", "topol.top", "-o", "ions.tpr", "-maxwarn", "3"]
    run_command(command)
    command = ["gmx", "genion", "-s", "ions.tpr", "-o", "structure_solv_ions.gro", "-p", "topol.top", 
               "-pname", "NA", "-nname", "CL", "-neutral", "-conc", "0.15", "-seed", "12345"]
    input_text = "14\n"
    run_command(command, input_text)
    command = ["gmx", "make_ndx", "-f", "structure_solv_ions.gro", "-o", "index.ndx"]
    input_text = "name 19 SOLV\n1 | 12\nname 20 SOLU\nq\n"
    run_command(command, input_text)

def get_pdb_files(pdb_directory, gpu_index, num_gpus):
    error_file_path = f"{pdb_directory}errors.txt"
    error_entries = set()
    if os.path.isfile(error_file_path):
        with open(error_file_path, "r") as error_file:
            error_entries = {line.strip() for line in error_file}
    # pdb_files = sorted([f for f in os.listdir(pdb_directory) if f.endswith('.pdb')])
    pdb_files = sorted([f for f in os.listdir(pdb_directory) if f.endswith('.pdb')])
    # completed_files = {os.path.splitext(f)[0] for f in os.listdir(os.path.join(pdb_directory, 'step5')) if f.endswith('.gro')}
    # pdb_files = [f for f in pdb_files if os.path.splitext(f)[0] not in completed_files and os.path.splitext(f)[0] not in {os.path.splitext(entry)[0] for entry in error_entries}]
    # pdb_files = [f for f in pdb_files if os.path.splitext(f)[0] not in completed_files]
    # total_rows = len(pdb_files)
    # portion_size = total_rows // num_gpus
    # start_idx = gpu_index * portion_size
    # end_idx = (gpu_index + 1) * portion_size if gpu_index < (num_gpus - 1) else total_rows
    # pdb_files = pdb_files[start_idx:end_idx]
    return pdb_files

pdb_files = get_pdb_files(pdb_directory, gpu_index, num_gpus)
print(f"Number of pdb_files to process: {len(pdb_files)}")



for input_pdb in pdb_files:
    if input_pdb != "ENSG00000005469.pdb":
        continue
    print(f"Current input_pdb: {input_pdb}")
    start_time = time.time()
    run_stucture_setup(os.path.join(pdb_directory, input_pdb))

    command = ["gmx", "grompp", "-v", "-f", f"step4.0_minimization.mdp", "-o", f"step4.0_minimization.tpr", 
               "-c", f"structure_solv_ions.gro", "-r", f"structure_solv_ions.gro", 
               "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_mini(command)
    command = ["gmx", "mdrun", "-v", "-deffnm", "step4.0_minimization", "-ntmpi", "1"]
    tries = 0
    while tries < 3 and not run_mini(command): ###
        tries += 1
    command_grompp = ["gmx", "grompp", "-f", f"step5_production.mdp", "-o", f"step5.tpr",
                      "-c", f"step4.0_minimization.gro", "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_command(command_grompp)
    command_mdrun = ["gmx", "mdrun", "-v", "-deffnm", "step5", "-ntmpi", "1"]
    run_command(command_mdrun)

    elapsed_time = time.time() - start_time

    if not os.path.isfile("step5.gro"):
        # with open(f"{pdb_directory}errors.txt", "a") as error_file:
        #     error_file.write(f"{input_pdb}\n")
        print(f"Process {input_pdb} failed in {elapsed_time:.2f} seconds.", file=sys.stderr)
    else:
        basename = os.path.splitext(os.path.basename(input_pdb))[0]
        # mv_command = ["mv", "step5.gro", f"{pdb_directory}step5/{basename}.gro"]
        # run_command(mv_command)
##################################################
        # mv_command = ["mv", "step4.0_minimization.gro", f"step4.0_minimization_1.gro"]
        # run_command(mv_command)
        # mv_command = ["mv", "step5.gro", f"step5_1.gro"]
        # run_command(mv_command)
#################################################
        print(f"Process {input_pdb} completed in {elapsed_time:.2f} seconds.", flush=True)
    rm_command = "rm step*.pdb"
    subprocess.run(rm_command, shell=True)













Number of pdb_files to process: 11294
Current input_pdb: ENSG00000005469.pdb


The largest distance between excluded atoms is 2.098 nm between atom 14346 and 14351
The largest distance between excluded atoms is 2.098 nm between atom
Step=    0, Dmax= 1.0e+05 nm, Epot=  6.03870e+16 Fmax=         inf, atom= 13790


13790


The largest distance between excluded atoms is 2.094 nm between atom 14346 and 14351
The largest distance between excluded atoms is 2.094 nm between atom
Process ENSG00000005469.pdb failed in 8.37 seconds.


In [ ]:
import subprocess
import os
import shutil
import time
import re
import sys


os.environ['PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/bin:' + os.environ['PATH']
if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib:' + os.environ['LD_LIBRARY_PATH']
else:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib'
os.environ['GMX_MAXBACKUP'] = '-1'
os.environ['GMX_MAXCONSTRWARN'] = '-1'

# if len(sys.argv) != 2:
#     print("Usage: python script.py <gpu_index>", flush=True)
#     sys.exit(1)
# gpu_index = int(sys.argv[1])
# pdb_directory = '/data/home/mrichte3/RNASeq/amide/'
gpu_index = 0
num_gpus = 6
pdb_directory = '/data/home/mrichte3/RNASeq/amide/'


def run_command(command, input_text=None, max_chars=100):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if "warning" in line.lower() or "fatal" in line.lower() or "random" in line.lower():
            print(line[:max_chars], file=sys.stderr)

def run_mini(command, input_text=None):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if ("steepest descents converged to" in line.lower() or
            "fatal" in line.lower() or
            "error" in line.lower() or
            "steepest descents did not converge" in line.lower()):
            print(line, file=sys.stderr)
            match = re.search(r'(\d+) steps', line)
            if match:
                steps = int(match.group(1))
                return steps == 5001
    return False
    
def run_stucture_setup(input_pdb):
    rm_command = "rm *.gro"
    subprocess.run(rm_command, shell=True)
    rm_command = "rm *.tpr"
    subprocess.run(rm_command, shell=True)
    command = ["gmx", "pdb2gmx", "-f", f"{input_pdb}", "-o", "structure_processed.gro", 
               "-p", "topol.top", "-i", "posre.itp"]
    input_text = "6\n1\n"        ############ 6 1 for custom
    run_command(command, input_text)
    command = ["gmx", "editconf", "-f", "structure_processed.gro", "-o", "structure_box.gro", "-c", "-d", "1.0", "-bt", "cubic"]
    run_command(command)
    command = ["gmx", "solvate", "-cp", "structure_box.gro", "-cs", "spc216.gro", "-o", "structure_solv.gro", "-p", "topol.top"]
    run_command(command)
    ###########################fails
    command = ["gmx", "grompp", "-f", "ions.mdp", "-c", "structure_solv.gro", "-p", "topol.top", "-o", "ions.tpr", "-maxwarn", "3"]
    run_command(command)
    command = ["gmx", "genion", "-s", "ions.tpr", "-o", "structure_solv_ions.gro", "-p", "topol.top", 
               "-pname", "NA", "-nname", "CL", "-neutral", "-conc", "0.15", "-seed", "12345"]
    input_text = "14\n"
    run_command(command, input_text)
    command = ["gmx", "make_ndx", "-f", "structure_solv_ions.gro", "-o", "index.ndx"]
    input_text = "name 19 SOLV\n1 | 12\nname 20 SOLU\nq\n"
    run_command(command, input_text)

def get_pdb_files(pdb_directory, gpu_index, num_gpus):
    error_file_path = f"{pdb_directory}errors.txt"
    error_entries = set()
    if os.path.isfile(error_file_path):
        with open(error_file_path, "r") as error_file:
            error_entries = {line.strip() for line in error_file}
    pdb_files = sorted([f for f in os.listdir(pdb_directory) if f.endswith('.pdb')])
    completed_files = {os.path.splitext(f)[0] for f in os.listdir(os.path.join(pdb_directory, 'step5')) if f.endswith('.gro')}
    pdb_files = [f for f in pdb_files if os.path.splitext(f)[0] not in completed_files and os.path.splitext(f)[0] not in {os.path.splitext(entry)[0] for entry in error_entries}]
    total_rows = len(pdb_files)
    portion_size = total_rows // num_gpus
    start_idx = gpu_index * portion_size
    end_idx = (gpu_index + 1) * portion_size if gpu_index < (num_gpus - 1) else total_rows
    pdb_files = pdb_files[start_idx:end_idx]
    return pdb_files

pdb_files = get_pdb_files(pdb_directory, gpu_index, num_gpus)
print(f"Number of pdb_files to process: {len(pdb_files)}")



for input_pdb in pdb_files:
    print(f"Current input_pdb: {input_pdb}")
    start_time = time.time()
    run_stucture_setup(os.path.join(pdb_directory, input_pdb))

    command = ["gmx", "grompp", "-v", "-f", f"step4.0_minimization.mdp", "-o", f"step4.0_minimization.tpr", 
               "-c", f"structure_solv_ions.gro", "-r", f"structure_solv_ions.gro", 
               "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_mini(command)
    command = ["gmx", "mdrun", "-v", "-deffnm", "step4.0_minimization", "-ntmpi", "1"]
    tries = 0
    while tries < 3 and not run_mini(command):
        tries += 1
    command_grompp = ["gmx", "grompp", "-f", f"step5_production.mdp", "-o", f"step5.tpr",
                      "-c", f"step4.0_minimization.gro", "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_command(command_grompp)
    command_mdrun = ["gmx", "mdrun", "-v", "-deffnm", "step5", "-ntmpi", "1"]
    run_command(command_mdrun)

    elapsed_time = time.time() - start_time

    if not os.path.isfile("step5.gro"):
        with open(f"{pdb_directory}errors.txt", "a") as error_file:
            error_file.write(f"{input_pdb}\n")
        print(f"Process {input_pdb} failed in {elapsed_time:.2f} seconds.", file=sys.stderr)
    else:
        basename = os.path.splitext(os.path.basename(input_pdb))[0]
        mv_command = ["mv", "step5.gro", f"{pdb_directory}step5/{basename}.gro"]
        run_command(mv_command)
        print(f"Process {input_pdb} completed in {elapsed_time:.2f} seconds.")
    rm_command = "rm step*.pdb"
    subprocess.run(rm_command, shell=True)
    # basename = os.path.splitext(os.path.basename(input_pdb))[0]
    # mv_command = ["mv", "step5.gro", f"/data/home/mrichte3/RNASeq/amide/step5/{basename}.gro"]
    # run_command(mv_command)












Setting the maximum number of constraint warnings to 2147483647


Process ENSG00000079150.pdb completed in 262.89 seconds.
Current input_pdb: ENSG00000079156.pdb


WARNING 1 [file topol.top, line 50]:
WARNING 2 [file topol.top, line 50]:
There were 2 WARNINGs
Using random seed 12345.
Steepest Descents did not converge to Fmax < 0 in 5001 steps.
WARNING 1 [file topol.top, line 52]:
There was 1 WARNING
Setting the maximum number of constraint warnings to 2147483647


Process ENSG00000079156.pdb completed in 203.53 seconds.
Current input_pdb: ENSG00000079215.pdb


WARNING 1 [file topol.top, line 50]:
WARNING 2 [file topol.top, line 50]:
There were 2 WARNINGs
Using random seed 12345.
Steepest Descents did not converge to Fmax < 0 in 5001 steps.
WARNING 1 [file topol.top, line 52]:
There was 1 WARNING
Setting the maximum number of constraint warnings to 2147483647


...

WARNING 1 [file topol.top, line 50]:
WARNING 2 [file topol.top, line 50]:
There were 2 WARNINGs
Using random seed 12345.
Steepest Descents converged to machine precision in 4839 steps,
Steepest Descents did not converge to Fmax < 0 in 5001 steps.
WARNING 1 [file topol.top, line 52]:
There was 1 WARNING
Setting the maximum number of constraint warnings to 2147483647


Process ENSG00000079999.pdb completed in 258.34 seconds.
Current input_pdb: ENSG00000080189.pdb


WARNING 1 [file topol.top, line 50]:
WARNING 2 [file topol.top, line 50]:
There were 2 WARNINGs
Using random seed 12345.
Steepest Descents converged to machine precision in 4018 steps,
Steepest Descents did not converge to Fmax < 0 in 5001 steps.
WARNING 1 [file topol.top, line 52]:
There was 1 WARNING
Setting the maximum number of constraint warnings to 2147483647


Process ENSG00000080189.pdb completed in 250.13 seconds.
Current input_pdb: ENSG00000080200.pdb


WARNING 1 [file topol.top, line 50]:
WARNING 2 [file topol.top, line 50]:
There were 2 WARNINGs
Using random seed 12345.
Steepest Descents converged to machine precision in 4670 steps,
Steepest Descents did not converge to Fmax < 0 in 5001 steps.
WARNING 1 [file topol.top, line 52]:
There was 1 WARNING


In [ ]:
### TEMPORARY SETUP, DON'T RUN
import subprocess
import os
import shutil
import time
import re
import sys
os.environ['PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/bin:' + os.environ['PATH']
if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib:' + os.environ['LD_LIBRARY_PATH']
else:
    os.environ['LD_LIBRARY_PATH'] = '/data/home/mrichte3/gromacs-2024.2/install/lib'
os.environ['GMX_MAXBACKUP'] = '-1'
os.environ['GMX_MAXCONSTRWARN'] = '-1'

def run_command(command, input_text=None, max_chars=100):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if "warning" in line.lower() or "fatal" in line.lower() or "random" in line.lower():
            print(line[:max_chars], file=sys.stderr)

def run_mini(command, input_text=None):
    result = subprocess.run(command, capture_output=True, text=True, input=input_text)
    output = result.stdout + result.stderr
    # print(output)
    for line in output.splitlines():
        if ("steepest descents converged to" in line.lower() or
            "fatal" in line.lower() or
            "error" in line.lower() or
            "steepest descents did not converge" in line.lower()):
            print(line, file=sys.stderr)
            match = re.search(r'(\d+) steps', line)
            if match:
                steps = int(match.group(1))
                return steps == 5001
    return False
    
def run_stucture_setup(input_pdb):
    rm_command = "rm *.gro"
    subprocess.run(rm_command, shell=True)
    rm_command = "rm *.tpr"
    subprocess.run(rm_command, shell=True)
    command = ["gmx", "pdb2gmx", "-f", f"{input_pdb}", "-o", "structure_processed.gro", 
               "-p", "topol.top", "-i", "posre.itp"]
    input_text = "6\n1\n"        ############ 6 1 for custom
    run_command(command, input_text)
    command = ["gmx", "editconf", "-f", "structure_processed.gro", "-o", "structure_box.gro", "-c", "-d", "1.0", "-bt", "cubic"]
    run_command(command)
    command = ["gmx", "solvate", "-cp", "structure_box.gro", "-cs", "spc216.gro", "-o", "structure_solv.gro", "-p", "topol.top"]
    run_command(command)
    ###########################fails
    command = ["gmx", "grompp", "-f", "ions.mdp", "-c", "structure_solv.gro", "-p", "topol.top", "-o", "ions.tpr", "-maxwarn", "3"]
    run_command(command)
    command = ["gmx", "genion", "-s", "ions.tpr", "-o", "structure_solv_ions.gro", "-p", "topol.top", 
               "-pname", "NA", "-nname", "CL", "-neutral", "-conc", "0.15", "-seed", "12345"]
    input_text = "14\n"
    run_command(command, input_text)
    command = ["gmx", "make_ndx", "-f", "structure_solv_ions.gro", "-o", "index.ndx"]
    input_text = "name 19 SOLV\n1 | 12\nname 20 SOLU\nq\n"
    run_command(command, input_text)

import os

def get_pdb_files(pdb_directory):
    process_number = 1
    error_file_path = "../amide_errors.txt"
    error_entries = set()
    if os.path.isfile(error_file_path):
        with open(error_file_path, "r") as error_file:
            error_entries = {line.strip() for line in error_file}
    
    pdb_files = sorted([f for f in os.listdir(pdb_directory) if f.endswith('.pdb')])
    completed_files = {os.path.splitext(f)[0] for f in os.listdir(os.path.join(pdb_directory, 'step5')) if f.endswith('.gro')}
    pdb_files = [f for f in pdb_files if os.path.splitext(f)[0] not in completed_files and os.path.splitext(f)[0] not in {os.path.splitext(entry)[0] for entry in error_entries}]
    print(f"Number of pdb_files to process before truncate: {len(pdb_files)}")
    truncate_file = 'ENSG00000105497.pdb'
    truncate_index = pdb_files.index(truncate_file) if truncate_file in pdb_files else len(pdb_files)
    pdb_files = pdb_files[:truncate_index]
    print(f"Number of pdb_files to process after truncate: {len(pdb_files)}")
    total_files = len(pdb_files)
    half_size = total_files // 2
    if process_number == 1:
        pdb_files = pdb_files[:half_size]
    elif process_number == 2:
        pdb_files = pdb_files[half_size:]
    print(f"Number of pdb_files to process: {len(pdb_files)}")
    return pdb_files

pdb_directory = '/data/home/mrichte3/RNASeq/amide/'
pdb_files = get_pdb_files(pdb_directory)

for input_pdb in pdb_files:
    print(f"Current input_pdb: {input_pdb}")
    start_time = time.time()
    run_stucture_setup(os.path.join(pdb_directory, input_pdb))

    command = ["gmx", "grompp", "-v", "-f", f"step4.0_minimization.mdp", "-o", f"step4.0_minimization.tpr", 
               "-c", f"structure_solv_ions.gro", "-r", f"structure_solv_ions.gro", 
               "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_mini(command)
    command = ["gmx", "mdrun", "-v", "-deffnm", "step4.0_minimization", "-ntmpi", "1"]
    tries = 0
    while tries < 3 and not run_mini(command):
        tries += 1
    command_grompp = ["gmx", "grompp", "-f", f"step5_production.mdp", "-o", f"step5.tpr",
                      "-c", f"step4.0_minimization.gro", "-p", "topol.top", "-n", "index.ndx", "-maxwarn", "5"]
    run_command(command_grompp)
    command_mdrun = ["gmx", "mdrun", "-v", "-deffnm", "step5", "-ntmpi", "1"]
    run_command(command_mdrun)

    elapsed_time = time.time() - start_time

    if not os.path.isfile("step5.gro"):
        with open("../amide_errors.txt", "a") as error_file:
            error_file.write(f"{input_pdb}\n")
        print(f"Process {input_pdb} failed in {elapsed_time:.2f} seconds.", file=sys.stderr)
    else:
        basename = os.path.splitext(os.path.basename(input_pdb))[0]
        mv_command = ["mv", "step5.gro", f"/data/home/mrichte3/RNASeq/amide/step5/{basename}.gro"]
        run_command(mv_command)
        print(f"Process {input_pdb} completed in {elapsed_time:.2f} seconds.")
    # basename = os.path.splitext(os.path.basename(input_pdb))[0]
    # mv_command = ["mv", "step5.gro", f"/data/home/mrichte3/RNASeq/amide/step5/{basename}.gro"]
    # run_command(mv_command)
    # rm_command = "rm step*.pdb"
    # subprocess.run(rm_command, shell=True)







